## Get the information of {Revenue, OperatingIncome, NetIncome} together

In [8]:
# -*- coding: utf-8 -*-
# pip install yahooquery pandas numpy
from yahooquery import Ticker
import pandas as pd
import numpy as np
from time import sleep

# ========== 1) 目标公司清单（可按需增删） ==========
JP_TICKERS = {
    "明豊エンタープライズ": "8927.T",
    "明和地所": "8869.T",
    "ゴールドクレスト": "8871.T",
    "エスリード": "8877.T",
    "フェイスネットワーク": "3489.T",
    "コロンビア・ワークス": "146A.T",     # 可能取不到数据，脚本会自动标记
    "プロパスト": "3236.T",
    "アーバネットコーポレーション": "3242.T",
    "セントラル総合開発": "3238.T",
    "ディア・ライフ": "3245.T",
    "コーセーアールイー": "3246.T",
    "グローバル・リンク・マネジメント": "3486.T",
    "霞ヶ関キャピタル": "3498.T",
    "山忠": "391A.T",                     # 可能取不到数据，脚本会自动标记
    "コスモスイニシア": "8844.T",
    "日神グループホールディングス": "8881.T",
    "シーラホールディングス": "8887.T",
    "日本エスコン": "8892.T",
}

# ========== 2) 抓取函数 ==========
def fetch_income_df(tickers, freq="q"):
    """
    使用 yahooquery 批量抓取损益表。
    freq: 'q' 为季度，'a' 为年度
    返回 DataFrame：index 含 symbol、asOfDate；列含 TotalRevenue/OperatingIncome/NetIncome 等
    """
    t = Ticker(list(tickers), asynchronous=True)
    df = t.income_statement(frequency=freq)  # 直接返回 DataFrame（yahooquery 会合并所有股票）
    # 兼容：若返回为空或不是 DataFrame
    if not isinstance(df, pd.DataFrame) or df.empty:
        return pd.DataFrame()
    # 有些版本 index 是普通列；统一保证 symbol & asOfDate 作为列存在
    if "symbol" not in df.columns and isinstance(df.index, pd.MultiIndex):
        df = df.reset_index()
    elif "symbol" not in df.columns:
        df = df.reset_index()
    return df

# ========== 3) 主流程：季度 + 年度 ==========
tickers_all = list(JP_TICKERS.values())

df_q = fetch_income_df(tickers_all, freq="q")
df_a = fetch_income_df(tickers_all, freq="a")

# 标准化列名（不同版本/公司可能有大小写差异，做一层容错）
def std_cols(df):
    if df.empty:
        return df
    cols = {c: c for c in df.columns}
    rename_map = {}
    for c in df.columns:
        lc = c.lower()
        if lc == "asofdate": rename_map[c] = "asOfDate"
        if lc == "periodtype": rename_map[c] = "periodType"
        if lc == "totalrevenue": rename_map[c] = "TotalRevenue"
        if lc == "operatingincome": rename_map[c] = "OperatingIncome"
        if lc == "netincome": rename_map[c] = "NetIncome"
        if lc == "currencycode": rename_map[c] = "currencyCode"
    if rename_map:
        df = df.rename(columns=rename_map)
    return df

df_q = std_cols(df_q)
df_a = std_cols(df_a)

# 只保留我们关心的核心列（存在就保留，不存在就跳过）
CORE_COLS = ["symbol", "asOfDate", "currencyCode", "TotalRevenue", "OperatingIncome", "NetIncome", "periodType"]
df_q = df_q[[c for c in CORE_COLS if c in df_q.columns]].copy() if not df_q.empty else pd.DataFrame()
df_a = df_a[[c for c in CORE_COLS if c in df_a.columns]].copy() if not df_a.empty else pd.DataFrame()

# 加入公司日文名（用映射反查）
ticker_to_name = {v: k for k, v in JP_TICKERS.items()}
for df in [df_q, df_a]:
    if not df.empty:
        df["CompanyNameJP"] = df["symbol"].map(ticker_to_name)
        # 排序更易读
        df.sort_values(["symbol", "asOfDate"], inplace=True)

# ========== 4) 审计：哪些代码成功/失败 ==========
def audit_availability(df, label):
    """返回每个 symbol 是否取到数据的表"""
    if df.empty:
        return pd.DataFrame(columns=["symbol", f"{label}_has_data"])
    g = (df.groupby("symbol").size()
           .rename(f"{label}_rows")
           .reset_index())
    g[f"{label}_has_data"] = g[f"{label}_rows"] > 0
    return g[["symbol", f"{label}_has_data"]]

aud_q = audit_availability(df_q, "quarterly")
aud_a = audit_availability(df_a, "annual")

audit = pd.merge(
    pd.DataFrame({"symbol": tickers_all}),
    aud_q, on="symbol", how="left"
)
audit = pd.merge(audit, aud_a, on="symbol", how="left")
audit["CompanyNameJP"] = audit["symbol"].map(ticker_to_name)
audit["quarterly_has_data"] = audit["quarterly_has_data"].fillna(False)
audit["annual_has_data"] = audit["annual_has_data"].fillna(False)

# ========== 5) 输出标准化长表：方便与你的 REMETIS 合并 ==========
def to_long(df, freq_label):
    """
    宽 -> 长：
    输出列：CompanyNameJP, symbol, asOfDate, Metric, Value, currencyCode, periodType, Freq
    Metric ∈ {Revenue, OperatingIncome, NetIncome}
    """
    if df.empty:
        return pd.DataFrame(columns=[
            "CompanyNameJP", "symbol", "asOfDate", "Metric", "Value", "currencyCode", "periodType", "Freq"
        ])
    slim = df.copy()
    # 兼容没有 periodType/currencyCode 的情况
    if "periodType" not in slim.columns: slim["periodType"] = np.nan
    if "currencyCode" not in slim.columns: slim["currencyCode"] = np.nan
    keep = ["CompanyNameJP", "symbol", "asOfDate", "currencyCode", "periodType",
            "TotalRevenue", "OperatingIncome", "NetIncome"]
    keep = [c for c in keep if c in slim.columns]
    slim = slim[keep]
    # 重命名为标准Metric
    rename_metric = {
        "TotalRevenue": "Revenue",
        "OperatingIncome": "OperatingIncome",
        "NetIncome": "NetIncome"
    }
    slim = slim.rename(columns=rename_metric)
    # 宽转长
    long = slim.melt(
        id_vars=[c for c in ["CompanyNameJP", "symbol", "asOfDate", "currencyCode", "periodType"] if c in slim.columns],
        value_vars=[c for c in ["Revenue", "OperatingIncome", "NetIncome"] if c in slim.columns],
        var_name="Metric", value_name="Value"
    )
    long["Freq"] = freq_label
    # 排序
    long = long.sort_values(["symbol", "asOfDate", "Metric"]).reset_index(drop=True)
    return long

long_q = to_long(df_q, "Quarterly")
long_a = to_long(df_a, "Annual")

# 合并两个频率
financials_long = pd.concat([long_q, long_a], ignore_index=True)

# ========== 6) 保存到 CSV ==========
financials_long.to_csv("financials_yq_core_metrics.csv", index=False, encoding="utf-8-sig")
audit.sort_values("symbol").to_csv("financials_yq_audit.csv", index=False, encoding="utf-8-sig")

print("✅ Saved:")
print(" - financials_yq_core_metrics.csv")
print(" - financials_yq_audit.csv")

# 提示哪些公司无数据
no_q = audit.loc[~audit["quarterly_has_data"], ["symbol", "CompanyNameJP"]]
no_a = audit.loc[~audit["annual_has_data"], ["symbol", "CompanyNameJP"]]
print("\n[No Quarterly Data]")
print(no_q.to_string(index=False))
print("\n[No Annual Data]")
print(no_a.to_string(index=False))


✅ Saved:
 - financials_yq_core_metrics.csv
 - financials_yq_audit.csv

[No Quarterly Data]
symbol CompanyNameJP
391A.T            山忠

[No Annual Data]
symbol CompanyNameJP
391A.T            山忠


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3428\489150770.py:109: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  audit["quarterly_has_data"] = audit["quarterly_has_data"].fillna(False)
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_3428\489150770.py:110: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  audit["annual_has_data"] = audit["annual_has_data"].fillna(False)


## Get specific Information of each company(Maybe better than the code above）

In [9]:
# -*- coding: utf-8 -*-
# pip install yahooquery pandas numpy
import os
import pandas as pd
import numpy as np
from yahooquery import Ticker
from time import sleep

# ===== 1) 目标公司清单（可增删）=====
JP_TICKERS = {
    "明豊エンタープライズ": "8927.T",
    "明和地所": "8869.T",
    "ゴールドクレスト": "8871.T",
    "エスリード": "8877.T",
    "フェイスネットワーク": "3489.T",
    "コロンビア・ワークス": "146A.T",   # 可能无数据
    "プロパスト": "3236.T",
    "アーバネットコーポレーション": "3242.T",
    "セントラル総合開発": "3238.T",
    "ディア・ライフ": "3245.T",
    "コーセーアールイー": "3246.T",
    "グローバル・リンク・マネジメント": "3486.T",
    "霞ヶ関キャピタル": "3498.T",
    "山忠": "391A.T",                   # 可能无数据
    "コスモスイニシア": "8844.T",
    "日神グループホールディングス": "8881.T",
    "シーラホールディングス": "8887.T",
    "日本エスコン": "8892.T",
}

# ===== 2) 工具函数 =====
def ensure_dir(path: str):
    if not os.path.exists(path):
        os.makedirs(path, exist_ok=True)

def to_df(obj):
    """把 yahooquery 返回的对象转为 DataFrame（若已是DF则直接返回）"""
    if isinstance(obj, pd.DataFrame):
        return obj.copy()
    try:
        return pd.DataFrame(obj)
    except Exception:
        return pd.DataFrame()

def normalize_df(df, symbol=None, company_name=None):
    """确保有 symbol 列、重置索引，附加公司名。"""
    if df is None or df.empty:
        return pd.DataFrame()
    if "symbol" not in df.columns:
        # 如果是 MultiIndex 或普通索引，把索引转列
        df = df.reset_index()
        if "symbol" not in df.columns and symbol is not None:
            df["symbol"] = symbol
    if company_name is not None and "CompanyNameJP" not in df.columns:
        df["CompanyNameJP"] = company_name
    # asOfDate/periodType 等列名大小写的容错
    rename_map = {}
    for c in df.columns:
        lc = c.lower()
        if lc == "asofdate": rename_map[c] = "asOfDate"
        if lc == "periodtype": rename_map[c] = "periodType"
        if lc == "currencycode": rename_map[c] = "currencyCode"
    if rename_map:
        df = df.rename(columns=rename_map)
    return df

# ===== 3) 抓取并保存（逐公司 & 汇总）=====
combined_all = []      # all_financial_data 的汇总
combined_inc_q = []    # income_statement 季度汇总
combined_inc_a = []    # income_statement 年度汇总
combined_bs_q  = []    # balance_sheet 季度汇总
combined_bs_a  = []    # balance_sheet 年度汇总

for name, sym in JP_TICKERS.items():
    print(f"==> Fetching: {name} ({sym})")
    out_dir = os.path.join(os.getcwd(), sym.replace(".", "_"))  # e.g., 8927_T
    ensure_dir(out_dir)

    try:
        t = Ticker(sym)

        # 3.1 all_financial_data
        afd = to_df(t.all_financial_data())
        afd = normalize_df(afd, symbol=sym, company_name=name)
        afd.to_csv(os.path.join(out_dir, "all_financial_data.csv"),
                   index=False, encoding="utf-8-sig")
        if not afd.empty:
            combined_all.append(afd)

        # 3.2 income_statement（季度）
        inc_q = t.income_statement(frequency="q")
        inc_q = to_df(inc_q)
        inc_q = normalize_df(inc_q, symbol=sym, company_name=name)
        inc_q.to_csv(os.path.join(out_dir, "income_statement_quarterly.csv"),
                     index=False, encoding="utf-8-sig")
        if not inc_q.empty:
            combined_inc_q.append(inc_q)

        # 3.3 income_statement（年度）
        inc_a = t.income_statement(frequency="a")
        inc_a = to_df(inc_a)
        inc_a = normalize_df(inc_a, symbol=sym, company_name=name)
        inc_a.to_csv(os.path.join(out_dir, "income_statement_annual.csv"),
                     index=False, encoding="utf-8-sig")
        if not inc_a.empty:
            combined_inc_a.append(inc_a)

        # 3.4 balance_sheet（季度）
        bs_q = t.balance_sheet(frequency="q")
        bs_q = to_df(bs_q)
        bs_q = normalize_df(bs_q, symbol=sym, company_name=name)
        bs_q.to_csv(os.path.join(out_dir, "balance_sheet_quarterly.csv"),
                    index=False, encoding="utf-8-sig")
        if not bs_q.empty:
            combined_bs_q.append(bs_q)

        # 3.5 balance_sheet（年度）
        bs_a = t.balance_sheet(frequency="a")
        bs_a = to_df(bs_a)
        bs_a = normalize_df(bs_a, symbol=sym, company_name=name)
        bs_a.to_csv(os.path.join(out_dir, "balance_sheet_annual.csv"),
                    index=False, encoding="utf-8-sig")
        if not bs_a.empty:
            combined_bs_a.append(bs_a)

        # 友好限速（可视网速调小）
        sleep(0.4)

    except Exception as e:
        print(f"[WARN] {sym} failed: {e}")
        continue

# ===== 4) 汇总落盘（可直接用于后续分析/merge）=====
def save_combined(dfs, fname):
    if not dfs:
        # 输出空壳，便于后续流程不报错
        pd.DataFrame().to_csv(fname, index=False, encoding="utf-8-sig")
        print(f"Saved empty {fname} (no data)")
    else:
        combined = pd.concat(dfs, ignore_index=True)
        # 统一排序更易读
        if "symbol" in combined.columns:
            sort_cols = [c for c in ["symbol", "asOfDate"] if c in combined.columns]
            if sort_cols:
                combined = combined.sort_values(sort_cols)
        combined.to_csv(fname, index=False, encoding="utf-8-sig")
        print(f"Saved {fname} ({len(combined)} rows)")

save_combined(combined_all, "combined_all_financial_data.csv")
save_combined(combined_inc_q, "combined_income_statement_quarterly.csv")
save_combined(combined_inc_a, "combined_income_statement_annual.csv")
save_combined(combined_bs_q,  "combined_balance_sheet_quarterly.csv")
save_combined(combined_bs_a,  "combined_balance_sheet_annual.csv")

print("\n✅ Done.")


==> Fetching: 明豊エンタープライズ (8927.T)
==> Fetching: 明和地所 (8869.T)
==> Fetching: ゴールドクレスト (8871.T)
==> Fetching: エスリード (8877.T)
==> Fetching: フェイスネットワーク (3489.T)
==> Fetching: コロンビア・ワークス (146A.T)
==> Fetching: プロパスト (3236.T)
==> Fetching: アーバネットコーポレーション (3242.T)
==> Fetching: セントラル総合開発 (3238.T)
==> Fetching: ディア・ライフ (3245.T)
==> Fetching: コーセーアールイー (3246.T)
==> Fetching: グローバル・リンク・マネジメント (3486.T)
==> Fetching: 霞ヶ関キャピタル (3498.T)
==> Fetching: 山忠 (391A.T)
==> Fetching: コスモスイニシア (8844.T)
==> Fetching: 日神グループホールディングス (8881.T)
==> Fetching: シーラホールディングス (8887.T)
==> Fetching: 日本エスコン (8892.T)
Saved combined_all_financial_data.csv (81 rows)
Saved combined_income_statement_quarterly.csv (122 rows)
Saved combined_income_statement_annual.csv (165 rows)
Saved combined_balance_sheet_quarterly.csv (55 rows)
Saved combined_balance_sheet_annual.csv (79 rows)

✅ Done.
